## Deploy token-classification models from HuggingFaceHub to AzureML Online Endpoints

This sample shows how to deploy `token-classification` models from the HuggingFaceHub to an online endpoint for inference. Learn more about `token-classification` task: https://huggingface.co/tasks/token-classification

A large set of models hosted on [Hugging Face Hub](https://huggingface.co/models) are available in the Hugging Face Hub collection in AzureML Model Catalog. This collection is powered by the Hugging Face Hub community registry. Integration with the AzureML Model Catalog enables seamless deployment of Hugging Face Hub models in AzureML. _todo: learn more link_

### Outline
* Set up pre-requisites.
* Pick a model to deploy.
* Deploy the model for real time inference.
* Try sample inference.
* Clean up resources.

### Set up pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `HuggingFaceHub` community registry

In [12]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
    ClientSecretCredential,
)
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

# connect to a workspace
workspace_ml_client = None
try:
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    workspace = workspace_ml_client.workspace_name
    resource_group = workspace_ml_client.resource_group_name
except Exception as ex:
    print(ex)
    # Enter details of your workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"
    workspace_ml_client = MLClient(
        credential, subscription_id, resource_group, workspace
    )
# Connect to the HuggingFaceHub registry
registry_ml_client = MLClient(credential, registry_name="HuggingFaceHub")
print(registry_ml_client)

We could not find config.json in: . or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.
MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7efc6f6d0b50>,
         subscription_id=d5210851-d1ca-44ac-8071-a0c7191e9631,
         resource_group_name=prod-azure-ml-registry,
         workspace_name=None)


### Pick a model to deploy

Open the Model Catalog in AzureML Studio and choose the Hugging Face Hub collection. Filter by the `fill-mask` task and search any specific models you are interested in. In this example, we use the `bert-base-uncased` model. If you plan to deploy a different model, replace the model name and version accordingly. 

In [4]:
model_name = "dslim-bert-base-ner"
foundation_model = registry_ml_client.models.get(model_name, version="15")
print(
    "\n\nUsing model name: {0}, version: {1}, id: {2} for inferencing".format(
        foundation_model.name, foundation_model.version, foundation_model.id
    )
)



Using model name: dslim-bert-base-ner, version: 15, id: azureml://registries/HuggingFaceHub/models/dslim-bert-base-ner/versions/15 for inferencing


### Deploy the model to an online endpoint
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model. Create an online endpoint and then create an online deployment. You need to specify the Virtual Machine instance or SKU when creating the deployment. You can find the optimal CPU or GPU SKU for a model by opening the quick deployment dialog from the model page in the AzureML Model Catalog. Specify the SKU in the `instance_type` input in deployment settings below.

Typically Online Endpoints require you to provide scoring script and a docker container image (through an AzureML environment), in addition to the model. You don't need to worry about them for HuggingFace Hub models available in AzureML Model Catalog because we have enabled 'no code deployments' for these models by packaging scoring script and container image along with the model.

Learn more about Online Endpoints: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-online-endpoints

In [5]:
import time, sys
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    OnlineRequestSettings,
)

# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
online_endpoint_name = "token-classification-" + str(timestamp)
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for " + foundation_model.name + ", for token-classification task",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [6]:
# create a deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
# online endpoints can have multiple deployments with traffic split or shadow traffic. Set traffic to 100% for demo deployment
endpoint.traffic = {"demo": 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

Check: endpoint token-classification-1684196348 exists


...........................................................................................................................

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://token-classification-1684196348.westus.inference.ml.azure.com/score', 'openapi_uri': 'https://token-classification-1684196348.westus.inference.ml.azure.com/swagger.json', 'name': 'token-classification-1684196348', 'description': 'Online endpoint for dslim-bert-base-ner, for token-classification task', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ea4faa5b-5e44-4236-91f6-5483d5b17d14/resourcegroups/kritifaujdar-rg/providers/microsoft.machinelearningservices/workspaces/kritifaujdar-ws/onlineendpoints/token-classification-1684196348', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ea4faa5b-5e44-4236-91f6-5483d5b17d14/providers/Microsoft.MachineLearningServices/locations/westus/mfeOperationsStatus/oe:ca131d77-08b6-4466-bf1d-fa4ff204bf31:4add5955-947b-4ae3-847d-62e8fa66af5e?api-version=2022-02-01-preview'}, 'print_as_yaml': Tru

### Try sample inference

Online endpoints expose a REST API that can be integrated into your applications. Learn how to fetch the scoring REST API and credentials for online endpoints here: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-authenticate-online-endpoint

In this example, we will use the Python SDK helper method to invoke the endpoint. 

In [41]:
# Get the model object from HuggingFaceHub. We can use it to check for sample test data
import urllib.request, json

raw_data = urllib.request.urlopen(
    "https://huggingface.co/api/models/" + foundation_model.tags["modelId"]
)

print("https://huggingface.co/api/models/" + foundation_model.tags["modelId"])
data = json.load(raw_data)
print(data)

https://huggingface.co/api/models/dslim/bert-base-NER
{'_id': '621ffdc136468d709f17a8e5', 'id': 'dslim/bert-base-NER', 'modelId': 'dslim/bert-base-NER', 'author': 'dslim', 'sha': '7a1d333eb0aadffc59fd1e4f56bfedf56b5028e4', 'lastModified': '2023-05-09T16:37:55.000Z', 'private': False, 'disabled': False, 'gated': False, 'pipeline_tag': 'token-classification', 'tags': ['pytorch', 'tf', 'jax', 'safetensors', 'bert', 'token-classification', 'en', 'dataset:conll2003', 'arxiv:1810.04805', 'transformers', 'license:mit', 'autotrain_compatible', 'has_space'], 'downloads': 2006954, 'library_name': 'transformers', 'mask_token': '[MASK]', 'widgetData': [{'text': 'My name is Wolfgang and I live in Berlin'}, {'text': 'My name is Sarah and I live in London'}, {'text': 'My name is Clara and I live in Berkeley, California.'}], 'likes': 166, 'model-index': None, 'config': {'architectures': ['BertForTokenClassification'], 'model_type': 'bert'}, 'cardData': {'language': 'en', 'datasets': ['conll2003'], 'li

In [10]:
# check if there is sample inference data available on HuggingFaceHub for the model, else try with the backup sample data
scoring_file = "./sample_score.json"
inputs = []
if "widgetData" in data:
    for input in data["widgetData"]:
        inputs.append(input["text"])
    # write the sample_score.json file
    score_dict = {"inputs": inputs}
    with open(scoring_file, "w") as outfile:
        json.dump(score_dict, outfile)
else:
    scoring_file = "./sample_score_backup.json"

# print the sample scoring file
print("\n\nSample scoring file: ")
with open(scoring_file) as json_file:
    scoring_data = json.load(json_file)
    print(scoring_data)



Sample scoring file: 
{'inputs': ['My name is Wolfgang and I live in Berlin', 'My name is Sarah and I live in London', 'My name is Clara and I live in Berkeley, California.']}


In [29]:
# score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="demo",
    request_file=scoring_file,
)
response_json = json.loads(response)
print(json.dumps(response_json, indent=2))


[
  [
    {
      "entity": "B-PER",
      "score": 0.9990139007568359,
      "index": 4,
      "word": "Wolfgang",
      "start": 11,
      "end": 19
    },
    {
      "entity": "B-LOC",
      "score": 0.9996449947357178,
      "index": 9,
      "word": "Berlin",
      "start": 34,
      "end": 40
    }
  ],
  [
    {
      "entity": "B-PER",
      "score": 0.9985477328300476,
      "index": 4,
      "word": "Sarah",
      "start": 11,
      "end": 16
    },
    {
      "entity": "B-LOC",
      "score": 0.9996215105056763,
      "index": 9,
      "word": "London",
      "start": 31,
      "end": 37
    }
  ],
  [
    {
      "entity": "B-PER",
      "score": 0.9964176416397095,
      "index": 4,
      "word": "Clara",
      "start": 11,
      "end": 16
    },
    {
      "entity": "B-LOC",
      "score": 0.9961979985237122,
      "index": 9,
      "word": "Berkeley",
      "start": 31,
      "end": 39
    },
    {
      "entity": "B-LOC",
      "score": 0.9990196228027344,
      "ind

In [18]:
# load the repsonse into a dataframe to better visualize the results
import pandas as pd

response_df = pd.DataFrame()
for result in response_json:
    for row in result:
        df = pd.DataFrame(row, index=[0])
        response_df = pd.concat([response_df, df], ignore_index=True)

print(response_df.head(len(response_df)))

  entity     score  index        word  start  end
0  B-PER  0.999014      4    Wolfgang     11   19
1  B-LOC  0.999645      9      Berlin     34   40
2  B-PER  0.998548      4       Sarah     11   16
3  B-LOC  0.999622      9      London     31   37
4  B-PER  0.996418      4       Clara     11   16
5  B-LOC  0.996198      9    Berkeley     31   39
6  B-LOC  0.999020     11  California     41   51


Let's make the output prettier and print it in below format:

"My name is Wolfgang [B-PER] and I live in Berlin [B-LOC]"

In [39]:
import json

def concatenate_entity(input_string, json_data):
    data = json.loads(json_data)
    output_string = input_string
    
    for sublist in data:
        for entity in sublist:
            word = entity.get("word")
            entity_type = entity.get("entity")
            if word and entity_type:
                if word in output_string:
                    output_string = output_string.replace(word, f"{word} [{entity_type}]")
    
    return output_string

for input_string in scoring_data['inputs']:
  output = concatenate_entity(input_string, response)
  print(output)

My name is Wolfgang [B-PER] and I live in Berlin [B-LOC]
My name is Sarah [B-PER] and I live in London [B-LOC]
My name is Clara [B-PER] and I live in Berkeley [B-LOC], California [B-LOC].


### Delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint.

In [40]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()

............................................................................................